In [1]:
import math

# Sound Pressure
A sound can be described by means of the time-varying sound pressure, $p(t)$. The unit for sound pressure is the pascal, $Pa$. 
<br>
<br>
The following sound pressure thresholds are relevant in psychoacoustics:
- Absolute Threshold: $10^{-5} Pa$
- Pain Threshold: $10^2 Pa$ 

The sound pressure level, $L$, is a logrithmic measure of sound pressure. The relationship between the sound pressure and the sound pressure level is as follows:
$$ L = 20 log \left( \frac{p}{p_0} \right) dB $$
where $p_0$ is the reference value of the sound pressure, standardized to:
$$ p_0 = 20 \mu Pa $$

In [2]:
def sound_pressure_level(p):
    """Calculate the sound pressure level

    Parameters
    ----------
    p : float
        sound pressure (Pa)

    """
    p0 = 20e-6
    return 20 * math.log10(p/p0)

# Sound Intensity
Sound intensity, $I$, is related to sound pressure level as follows:
$$ L = 20 log \left( \frac{p}{p_0} \right) dB = 10 log \left( \frac{I}{I_0} \right) dB$$
where $I_0$ is the reference value of sound intensity, standardized to:
$$ I_0 = 10^{-12} \frac{W}{m^2} $$

In [ ]:
def sound_intensity_level(I):
    """Calculate the sound intensity level

    Parameters
    ----------
    I : float
        sound intensity (W/m^2)

    """
    I0 = 10e-12
    return 10 * math.log10(I/I0)

# Sound Density
Sound density is a measure of the sound intensity within a bandwidth of 1 Hz. It is useful measure when dealing with noises. The sound density is sometimes referred to as "noise power density", although this is not quite correct. The sound density level, $l$, is a logrithmic measure of sound density. It can be calculated as follows:
$$ l = L - 10 log \left( \frac{\Delta f}{Hz} \right) dB $$
where $\Delta f$ represents the bandwidth of the sound in question measured in Hz.

In [ ]:
def sound_density_level(L, delta_f):
    """Calculate the sound density level

    Parameters
    ----------
    L : float
        sound pressure level or sound intensity level (dB)
    delta_f : float
        bandwidth (Hz)

    """
    return L - 10 * math.log10(delta_f)